In [1]:
import numpy as np
import sympy as sp
from numba import njit,jit,prange

$$
\begin{array}{ll}\left|S_{2 n}-S_{n}\right|<\varepsilon & \left|S_{2 n}\right| \leq 1 
\\
\left|\frac{S_{2 n}-S_{n}}{S_{2 n}}\right|<\varepsilon & \left|S_{2 n}\right|>1\end{array}
$$

## 变步长梯形积分
$$
\begin{array}{c}T_{1}=\frac{h}{2}\left[f\left(x_{k}\right)+f\left(x_{k+1}\right)\right] 
\\
T_{2}=\frac{h}{4}\left[f\left(x_{k}\right)+2 f\left(x_{k+\frac{1}{2}}\right)+f\left(x_{k+1}\right)\right]\end{array}
\\
\left\{\begin{array}{l}T_{2 n}=\frac{1}{2} T_{n}+\frac{h}{2} \sum_{k=0}^{n-1} f\left(x_{k+\frac{1}{2}}\right) \\ x_{k+\frac{1}{2}}=a+\left(k+\frac{1}{2}\right) h\end{array}\right.
$$

## 变步长辛普森积分
$$
S_{n}=\frac{4}{3} T_{2 n}-\frac{1}{3} T_{n}=T_{2 n}+\frac{1}{3}\left(T_{2 n}-T_{n}\right)
$$

In [2]:
# 变步长辛普森积分
def step_change_simpson_int(int_fun, s,d, nn, ep):
    error = 10.
    h = (d-s)/nn

    # 1分梯形积分
    sum_f1 = lambda x:(int_fun(x)+int_fun(x+h))*h/2
    t1 = np.sum( sum_f1(np.arange(s, d-h, h)) )
    # 2分梯形积分
    sum_f = lambda x:int_fun(x+h/2)
    t2 = t1/2 + h/2 *np.sum(sum_f(np.arange(s, d-h, h)+h/2))
    # 1分辛普森积分
    s1 = t2 + 1/3*(t2-t1)
    
    while error>=ep:
        h/=2
        # 2分梯形积分
        sum_f1 = lambda x:(int_fun(x)+int_fun(x+h))*h/2
        t1 = np.sum( sum_f1(np.arange(s, d-h, h)) )
        # 4分梯形积分
        sum_f = lambda x:int_fun(x+h/2)
        t2 = t1/2 + h/2 *np.sum(sum_f(np.arange(s, d-h, h)+h/2))
        # 2分辛普森积分
        s2 = t2 + 1/3*(t2-t1)
        
        # 计算误差
        if(np.abs(s2)<=1):
            # 计算绝对误差
            error = np.abs(s2 - s1)
        else:
            # 计算相对误差
            error = np.abs((s2-s1)/s2)
        s1 = s2
    
    return s2

# 变步长梯形积分法:被积函数，积分下限，积分上限，网格数
def step_change_ladder_int(int_fun, s,d, nn, ep):
    t1 = 0.
    error = 10.
    # h为步长
    h = (d-s)/nn
    # 1分积分
    sum_f1 = lambda x:(int_fun(x)+int_fun(x+h))*h/2
    t1 = np.sum( sum_f1(np.arange(s, d-h, h)) )

    while error>=ep:
        # 2分积分
        sum_f2 = lambda x:int_fun(x+h/2)
        t2 = t1/2 + h/2 *np.sum(sum_f2(np.arange(s, d-h, h)+h/2))
        # 计算误差
        if(np.abs(t2)<=1):
            # 计算绝对误差
            error = np.abs(t2 - t1)
        else:
            # 计算相对误差
            error = np.abs((t2-t1)/t2)
        h /= 2
        t1 = t2
    
    return t2

# step_change_ladder_int(lambda x:np.cos(1/(x**2+1)), 0., np.pi, 100, 1e-5)
# step_change_simpson_int(lambda x:np.cos(1/(x**2+1)), 0., np.pi, 100, 1e-5)

$$
s=4 a \int_{0}^{\pi / 2} \sqrt{1-\left(\frac{c}{a}\right)^{2} \sin ^{2} \theta} d \theta
$$

In [7]:
def elliptic_perimeter(theta):
    return 4*7882.5*np.sqrt(1.-(972.5/7782.5)**2)*np.sin(theta)**2


# 符号解
theta = sp.symbols(r'\theta')
sym_int = sp.integrate(4*7882.5*sp.sqrt(1.-(972.5/7782.5)**2)*sp.sin(theta)**2, [theta,0,np.pi/2])
print("符号解",sym_int)
# 数值解
la_int = step_change_ladder_int(elliptic_perimeter, 0., np.pi/2, 100, 1e-5)
simp_int = step_change_simpson_int(elliptic_perimeter, 0., np.pi/2, 100, 1e-5)

print("ep",1e-5)
print("la_int:",la_int,"error",np.abs(sym_int-la_int))
print("simpson:",simp_int,"error",np.abs(sym_int-simp_int))

la_int = step_change_ladder_int(elliptic_perimeter, 0., np.pi/2, 100, 1e-7)
simp_int = step_change_simpson_int(elliptic_perimeter, 0., np.pi/2, 100, 1e-7)
print("ep",1e-7)
print("la_int:",la_int,"error",np.abs(sym_int-la_int))
print("simpson:",simp_int,"error",np.abs(sym_int-simp_int))

符号解 24569.5020047926
ep 1e-05
la_int: 24569.262071949404 error 0.239932843196584
simpson: 24569.342047097263 error 0.159957695337653
ep 1e-07
la_int: 24569.500598928837 error 0.00140586376437568
simpson: 24569.50075512311 error 0.00124966949078953
